# The Dataset

We used this dataset for our miniproject: https://www.kaggle.com/datasets/puneet6060/intel-image-classification

The dataset contains 6 classes of images: `building`, `forest`, `glacier`, `mountain`, `sea`, and `street`. We decided to delete the class `street` and it's corresponding pictures to simplify the learning process. The images are divided into folders with their respective labels. We used the `image_dataset_from_directory` method of Keras to convert the images into a TensorFlow dataset object for training.

## Importing the Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
training_img_path = "./dataset/seg_train/seg_train/"

img_size = 50
batch_size = 32
seed = 42   # the seed will make sure the two datasets are not overlapping

train_ds = keras.utils.image_dataset_from_directory(
    training_img_path,
    validation_split=0.2,
    subset="training",
    labels="inferred",
    label_mode='int',
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

val_ds = keras.utils.image_dataset_from_directory(
    training_img_path,
    validation_split=0.2,
    subset="validation",
    labels="inferred",
    label_mode='int',
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

print(train_ds.class_names)

Let's see the first few images of the training dataset. Here, we'll define a function that we'll use again later in the notebook.

To ensure the model won't eat up too many of our computer's resources, we downscaled the images from $150 \times 150px$ to $50 \times 50px$.

In [ ]:
def show_dataset():
  plt.figure(figsize=(5, 5))
  for images, labels in train_ds.take(1):
    for i in range(12):
      ax = plt.subplot(3, 4, i + 1)
      plt.imshow(images[i].numpy().astype("uint8"))
      plt.title(train_ds.class_names[labels[i]])
      plt.axis("off")

show_dataset()

# Underfitting Model
We will create a model that is too simple and won't describe the data accurately enough.

## Building the Model
We start with a convolutional layer that'll import/rescale the images to `img_size`, using the ReLU activation function (for all the convolution and dense layers).

Then, run a kernel of $3 \times 3$ over each image 16 times. `padding=same` is referring to the padding of the image (needed because of the kernel) being filled with zeros.

Following that is another convolution layer and max pooling layer. Next, the image will be flattened into a vector, ready to be fed to the following dense layer. In this case, we first wanted to apply dropout to the layer, but we ended up just halfing the amount of nodes of that layer. Why? We learned that the difference between applying `dropout(0.5)` and halving the nodes of the layer is that dropout randomly drops out nodes during each training iteration, which means that **different** nodes will be dropped out in each iteration. This allows the network to learn more robust and generalizable representations of the data, as it is forced to rely on a subset of nodes in each iteration, which prevents overfitting. While halving the nodes reduces the number of nodes in the layer permanently, which means that the network has less capacity to learn and represent complex patterns in the data. This then leads to underfitting, where the model is not able to capture the important features in the data. Which is exactly what we want. Also, dropout is present during training, but not during inference.

The last dense layer represents the output layer, having a shared softmax activation layer to determine the probabilities of the 5 different classes.

This task was interesting because we initially built a model that was too simple, even for underfitting. After the second epoch, it couldn't learn any more information because it simply was not complex enough. As a result, the accuracy stalled at around 0.25 and stayed the same for all the remaining epochs. We were not happy with that, so we now created a model that can actually improve with each epoch while still underfitting.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(6, (3,3), input_shape=(img_size,img_size,3), activation='relu', padding='same'),
    MaxPooling2D(),

    Conv2D(6, (3,3), activation='relu', padding='same'),
    MaxPooling2D(),

    Flatten(),
    Dense(5, activation='softmax')
])
model._name='underfitting_model'

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

## Training the Model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

## Visualize and analyze the model
Now, we'll have a look at how well the model performs.

In [ ]:
def visualize_loss_and_accuracy():
    # Plot the training and validation Loss
    plt.figure(figsize=(10,4))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(['Train','Val'], loc= 'upper left')
    plt.show()

    # Plot the model accuracy
    plt.figure(figsize=(10,3))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

visualize_loss_and_accuracy()

As we see here, in the beginning, there's a gap in between the the validation and training loss and also in the accuracy. That is because of the randomness of the two different sample sets.

We can see from the learning curves that the model slowly converges and is indeed underfitting. Both the training and validation accuracy curves plateau at a low value, indicating that the model is not learning the patterns in the data well enough.

### Make Prediction on the test dataset

In [ ]:
testing_img_path = "./dataset/seg_test/seg_test/"

test_ds = keras.utils.image_dataset_from_directory(
    testing_img_path,
    labels="inferred",
    label_mode='int',
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

predictions = model.predict(test_ds)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.concatenate([y for x, y in test_ds], axis=0)

print('Predicted classes:', predicted_classes)
print('True labels:', true_classes)

### Evaluate the performance of the model

In [ ]:
def evaluate_model():
    accuracy = model.evaluate(test_ds)[1]
    print(f'Accuracy: {accuracy}')

evaluate_model()

### Classification Report


In [ ]:
test_ds.as_numpy_iterator

print('Classification Report:\n', metrics.classification_report(y_true=true_classes, y_pred=predicted_classes))

#https://www.kaggle.com/code/avantikab/intel-image-classification-cnn


### Confusion Matrix
The confusion matrix is another way to visualize the performance of the model and wether it is underfitting or not.

In [ ]:

def print_confusion_matrix():
    confusion_matrix = metrics.confusion_matrix(y_true=true_classes, y_pred=predicted_classes)
    display = metrics.ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=test_ds.class_names) #important, what mr lehmann said
    display.plot()
    plt.show()

print_confusion_matrix()


There is no clear diagonal line in the middle, in fact, the matrix shows that the mapping of the classes is all over the place. Which again confirms that this model is indeed underfitting.